# 타이타닉 생존자 예측

### 프로젝트 개요
건조 당시 세계 최대의 여객선이었지만,1912년의 최초이자 최후의 항해 때 빙산과 충돌해 침몰한 비운의 여객선. 아마도 세상에서 가장 유명한 여객선이자 침몰선일 것입니다.

침몰한 지 100년이 넘었지만 아직까지 세계에서 가장 유명한 침몰선입니다.

사망자 수는 1위는 아니지만, 세계적으로 유명한 영화의 영향도 있고, 당시 최첨단 기술에 대해 기대감이 컸던 사회에 큰 영향을 끼치기도 한데다가, 근대 사회에서 들어서자마자 얼마 안된, 그리고 유명인사들이 여럿 희생된 대참사이기 때문에 가장 유명한 침몰선이 되었습니다. 또한 이 사건을 기점으로 여러가지 안전 조약들이 생겨났으니 더더욱 그렇습니다.

### 데이터 설명
**파일 설명**
- train.csv : 생존 여부가 포함된 학습 데이터  
- test.csv : 생존 여부가 포함되지 않은 평가 데이터
- submission.csv : 평가 데이터(test.csv)를 이용하여 예측한 결과를 제출하기 위한 파일 (EX: OMR 카드)
**컬럼(columns) 설명**
- PassengerId: 승객 번호
- **Survivied: 생존여부 (1: 생존, 0: 사망)**
- Pclass: 좌석 등급 (1등급, 2등급, 3등급)
- Name: 승객명
- Sex: 성별
- age: 나이
- Sibsp: 형제 + 배우자 수
- Parch: 부모 + 자녀 수
- Ticket: 티켓번호
- Fare: 좌석 요금
- Cabin: 캐빈 번호
- Embarked: 탑승 항구 (S, C, Q)

### 진행 순서
- 학습 및 평가 데이터 로드
- 학습 데이터 분석
- 평가 데이터 예측
- 제출 및 점수확인

## 패키치 설치

In [7]:
!pip3 install mySUNI

In [19]:
import numpy as np
import pandas as pd
import seaborn as sns
from mySUNI import cds
from IPython.display import Image

## 데이터셋 다운로드

In [20]:
cds.download_project('타이타닉 생존자 예측', '1차수 B반', 'yong19@sk.com') 
# 차수정보의 경우 강사님께 문의해주세요.
# 이메일의 경우 @sk.com 형식의 이메일을 입력해주세요.


프로젝트: 타이타닉 생존자 예측
파일 목록

test.csv
- data/타이타닉 생존자 예측/test.csv

submission.csv
- data/타이타닉 생존자 예측/submission.csv

train.csv
- data/타이타닉 생존자 예측/train.csv



## 1) 학습 및 평가 데이터 로드

In [21]:
import os
print(os.getcwd())

/Users/yong19/Desktop/python19/first-project-1/study/mysuni/pandas/workshop/mySUNI-WorkShop-03-타이타닉 생존자 분석/workshop/mySUNI-WorkShop-04-타이타닉 생존자 예측


In [23]:
train = pd.read_csv('data/타이타닉 생존자 예측/train.csv')
test = pd.read_csv('data/타이타닉 생존자 예측/test.csv')
submission = pd.read_csv('data/타이타닉 생존자 예측/submission.csv')

In [24]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [25]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## 2) 학습 데이터 분석
**tip**
- train 데이터로 분석을 진행하여 생존율이 높게 나오는 조건을 찾습니다.
- test 데이터로 찾은 조건을 적용합니다.(Survived 컬럼에 적용)

In [26]:
# 탑승항구에 따른 생존율 비교
train.groupby('Embarked')['Survived'].mean()

Embarked
C    0.553571
Q    0.389610
S    0.336957
Name: Survived, dtype: float64

## 3) 평가 데이터 예측
#### 분석 결과
- 탑승 항구가 'C' 인경우 생존율 55%
- 탑승 항구가 'Q' 인경우 생존율 38%
- 탑승 항구가 'S' 인경우 생존율 3%

#### 분석 결과 적용
- test 데이터 프레임(평가 데이터)에서 탑승항구가 'C'이라면 Survived 컬럼의 값을 1(생존)으로 'Q' 또는 'S'라면 0(사망)으로 지정

In [29]:
test.loc[test['Embarked'] == 'C', 'Survived'] = 1
test.loc[test['Embarked'] == 'Q', 'Survived'] = 0
test.loc[test['Embarked'] == 'S', 'Survived'] = 0

## 4) 제출 및 점수확인

### 제출 파일 생성

In [30]:
submission['Survived'] = test['Survived'] # 예측 결과를 제출 파일에 복사
submission # 제출파일 확인

,PassengerId,Survived
0,892,0.0
1,893,0.0
2,894,0.0
3,895,0.0
4,896,0.0
...,...,...
413,1305,0.0
414,1306,1.0
415,1307,0.0
416,1308,0.0


### 결과 제출 및 점수 확인

In [31]:
cds.submit(submission)

파일을 저장하였습니다. 파일명: submission-14-40-20.csv
제출 여부 :success
오늘 제출 횟수 : 56
제출 결과:0.6507177033492823


# 제출 결과(정확도)를 높여보세요!

In [36]:
test['Survived'] = 0  #전체다 0 으로 주는것

In [37]:
submission['Survived'] = test['Survived'] # 예측 결과를 제출 파일에 복사
submission
cds.submit(submission)

파일을 저장하였습니다. 파일명: submission-14-42-06.csv
제출 여부 :success
오늘 제출 횟수 : 58
제출 결과:0.6220095693779905
